# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f07f489aa00>
├── 'a' --> tensor([[ 2.4528, -0.2200,  0.1426],
│                   [-1.0750,  1.1667,  0.5802]])
└── 'x' --> <FastTreeValue 0x7f07f489afd0>
    └── 'c' --> tensor([[-0.7945,  0.0770,  0.3646,  0.1818],
                        [-0.5019, -0.2319, -2.3030, -0.0652],
                        [-1.5730, -0.2927, -0.7706, -1.1081]])

In [4]:
t.a

tensor([[ 2.4528, -0.2200,  0.1426],
        [-1.0750,  1.1667,  0.5802]])

In [5]:
%timeit t.a

47.7 ns ± 0.298 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f07f489aa00>
├── 'a' --> tensor([[-0.2828,  0.1338,  0.4915],
│                   [ 1.2147, -0.4817,  0.2909]])
└── 'x' --> <FastTreeValue 0x7f07f489afd0>
    └── 'c' --> tensor([[-0.7945,  0.0770,  0.3646,  0.1818],
                        [-0.5019, -0.2319, -2.3030, -0.0652],
                        [-1.5730, -0.2927, -0.7706, -1.1081]])

In [7]:
%timeit t.a = new_value

54.9 ns ± 0.214 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 2.4528, -0.2200,  0.1426],
               [-1.0750,  1.1667,  0.5802]]),
    x: Batch(
           c: tensor([[-0.7945,  0.0770,  0.3646,  0.1818],
                      [-0.5019, -0.2319, -2.3030, -0.0652],
                      [-1.5730, -0.2927, -0.7706, -1.1081]]),
       ),
)

In [10]:
b.a

tensor([[ 2.4528, -0.2200,  0.1426],
        [-1.0750,  1.1667,  0.5802]])

In [11]:
%timeit b.a

44.7 ns ± 0.251 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.5917, -0.5303,  0.5215],
               [ 1.4566, -0.7730,  0.3872]]),
    x: Batch(
           c: tensor([[-0.7945,  0.0770,  0.3646,  0.1818],
                      [-0.5019, -0.2319, -2.3030, -0.0652],
                      [-1.5730, -0.2927, -0.7706, -1.1081]]),
       ),
)

In [13]:
%timeit b.a = new_value

383 ns ± 2.21 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

637 ns ± 3.02 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

8.53 µs ± 33.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

130 µs ± 769 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

128 µs ± 1.67 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f070b9efd30>
├── 'a' --> tensor([[[ 2.4528, -0.2200,  0.1426],
│                    [-1.0750,  1.1667,  0.5802]],
│           
│                   [[ 2.4528, -0.2200,  0.1426],
│                    [-1.0750,  1.1667,  0.5802]],
│           
│                   [[ 2.4528, -0.2200,  0.1426],
│                    [-1.0750,  1.1667,  0.5802]],
│           
│                   [[ 2.4528, -0.2200,  0.1426],
│                    [-1.0750,  1.1667,  0.5802]],
│           
│                   [[ 2.4528, -0.2200,  0.1426],
│                    [-1.0750,  1.1667,  0.5802]],
│           
│                   [[ 2.4528, -0.2200,  0.1426],
│                    [-1.0750,  1.1667,  0.5802]],
│           
│                   [[ 2.4528, -0.2200,  0.1426],
│                    [-1.0750,  1.1667,  0.5802]],
│           
│                   [[ 2.4528, -0.2200,  0.1426],
│                    [-1.0750,  1.1667,  0.5802]]])
└── 'x' --> <FastTreeValue 0x7f070b9efb50>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

24.8 µs ± 278 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f070b9c1fa0>
├── 'a' --> tensor([[ 2.4528, -0.2200,  0.1426],
│                   [-1.0750,  1.1667,  0.5802],
│                   [ 2.4528, -0.2200,  0.1426],
│                   [-1.0750,  1.1667,  0.5802],
│                   [ 2.4528, -0.2200,  0.1426],
│                   [-1.0750,  1.1667,  0.5802],
│                   [ 2.4528, -0.2200,  0.1426],
│                   [-1.0750,  1.1667,  0.5802],
│                   [ 2.4528, -0.2200,  0.1426],
│                   [-1.0750,  1.1667,  0.5802],
│                   [ 2.4528, -0.2200,  0.1426],
│                   [-1.0750,  1.1667,  0.5802],
│                   [ 2.4528, -0.2200,  0.1426],
│                   [-1.0750,  1.1667,  0.5802],
│                   [ 2.4528, -0.2200,  0.1426],
│                   [-1.0750,  1.1667,  0.5802]])
└── 'x' --> <FastTreeValue 0x7f070b9c1dc0>
    └── 'c' --> tensor([[-0.7945,  0.0770,  0.3646,  0.1818],
                        [-0.5019, -0.2319, -2.3030, -0.0652],
                 

In [23]:
%timeit t_cat(trees)

23 µs ± 86.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

50.7 µs ± 1.72 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.7945,  0.0770,  0.3646,  0.1818],
                       [-0.5019, -0.2319, -2.3030, -0.0652],
                       [-1.5730, -0.2927, -0.7706, -1.1081]],
              
                      [[-0.7945,  0.0770,  0.3646,  0.1818],
                       [-0.5019, -0.2319, -2.3030, -0.0652],
                       [-1.5730, -0.2927, -0.7706, -1.1081]],
              
                      [[-0.7945,  0.0770,  0.3646,  0.1818],
                       [-0.5019, -0.2319, -2.3030, -0.0652],
                       [-1.5730, -0.2927, -0.7706, -1.1081]],
              
                      [[-0.7945,  0.0770,  0.3646,  0.1818],
                       [-0.5019, -0.2319, -2.3030, -0.0652],
                       [-1.5730, -0.2927, -0.7706, -1.1081]],
              
                      [[-0.7945,  0.0770,  0.3646,  0.1818],
                       [-0.5019, -0.2319, -2.3030, -0.0652],
                       [-1.5730, -0.2927, -0.7706, -1.1081]],

In [26]:
%timeit Batch.stack(batches)

63.3 µs ± 293 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.7945,  0.0770,  0.3646,  0.1818],
                      [-0.5019, -0.2319, -2.3030, -0.0652],
                      [-1.5730, -0.2927, -0.7706, -1.1081],
                      [-0.7945,  0.0770,  0.3646,  0.1818],
                      [-0.5019, -0.2319, -2.3030, -0.0652],
                      [-1.5730, -0.2927, -0.7706, -1.1081],
                      [-0.7945,  0.0770,  0.3646,  0.1818],
                      [-0.5019, -0.2319, -2.3030, -0.0652],
                      [-1.5730, -0.2927, -0.7706, -1.1081],
                      [-0.7945,  0.0770,  0.3646,  0.1818],
                      [-0.5019, -0.2319, -2.3030, -0.0652],
                      [-1.5730, -0.2927, -0.7706, -1.1081],
                      [-0.7945,  0.0770,  0.3646,  0.1818],
                      [-0.5019, -0.2319, -2.3030, -0.0652],
                      [-1.5730, -0.2927, -0.7706, -1.1081],
                      [-0.7945,  0.0770,  0.3646,  0.1818],
                   

In [28]:
%timeit Batch.cat(batches)

119 µs ± 606 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

278 µs ± 2.47 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
